In [1]:
import os
import sys

from pathlib import Path


# si aggiunge al path la cartella utils per avere visibilità del module
path = Path(os.getcwd()).parent.parent
module_path = os.path.join(path, "detectwins")

sys.path.append(module_path)

In [2]:
import os
import torch
import numpy as np
import pandas as pd
import utils
import utils.datasets as datasets

from torch import nn
from tqdm import tqdm
from skimage import io
from random import sample
from sklearn.metrics import confusion_matrix, average_precision_score

In [3]:
artifact_path = os.path.join(path, "artifact")
project_path = os.path.join(path, "detectwins")

fake_dataset = "taming_transformer"
real_dataset = "coco"

# directory da dove vengono prelevate le immagini per il testing (devono essere nello spettro di fourier)
fake_data_dir = os.path.join(project_path, "temp", fake_dataset + "+" + real_dataset, "test", fake_dataset)
real_data_dir = os.path.join(project_path, "temp", fake_dataset + "+" + real_dataset, "test", real_dataset)

In [6]:
# config
device="cuda"

In [7]:
model = utils.ApnModel()

# per processare le immagini in scala di grigi per fare fourier serve una CNN 2D
model.efficientnet.conv_stem = nn.Conv2d(1, 32, 3, 2, 1, bias=False)

model.to(device);

In [ ]:
# per ricaricare il modello già allenato e il dataset di encoding (che servirà per il testing)
model.load_state_dict(torch.load("best_model.pt"))

In [47]:
# su quante immagini deve essere condotto il test
test_size = 1000

# si prelevano le immagini per fare il test
fake_images_paths = utils.get_file_paths(fake_data_dir)
real_images_paths = utils.get_file_paths(real_data_dir)

fake_images = sample(fake_images_paths, test_size)
real_images = sample(real_images_paths, test_size * 2)

In [48]:
if not os.path.isdir(os.path.join("..", "temp")):
    datasets.fourier(fake_dataset, real_dataset, artifact_path)

In [49]:
# cartelle dove prelevare le immagini del dataset di training 
fake_df_out_dir = os.path.join("..", "temp", "taming_transformer+coco", "train", "taming_transformer")
real_df_out_dir = os.path.join("..", "temp", "taming_transformer+coco", "train", "coco")

# si usa il dataset di training per creare il dataset di encodings che serve per il testing
df_out_path = os.path.join("..", "datasets", "fourier_out.csv")
df_out = pd.read_csv(df_out_path)

In [50]:
# si creano gli embeddings che vengono memorizzati 
if not os.path.isfile("database.csv"):
    df_enc = utils.get_encoding_csv(model, df_out["Anchor"], fake_df_out_dir, real_df_out_dir, device)
    df_enc.to_csv("database.csv", index=False)

else: 
    df_enc = pd.read_csv("database.csv")

In [51]:
y_true = []
y_pred = []

In [ ]:
# testo i fake
for i in tqdm(fake_images, desc="testing on fake images..."):
    path = i

    # si legge l'immagine
    img = io.imread(path)
    # si ottiene il vettore di embeddings dell'immagine
    img_enc = utils.get_image_embeddings(img, model, device)
    # si cerca nel dataset con gli encodings un'immagine simile 
    closest_label = utils.search_in_database(img_enc, df_enc)
    
    # se nel path dell'immagine c'è il nome del dataset real è real
    if real_dataset in str(closest_label):
        y_pred.append("real")
    # viceversa
    else:
        y_pred.append("fake")

In [ ]:
# testo i real
for i in tqdm(real_images, desc="testing on real images..."):
    path = i

    img = io.imread(path)
    img_enc = utils.get_image_embeddings(img, model, device)
    closest_label = utils.search_in_database(img_enc, df_enc)

    if real_dataset in str(closest_label):
        y_pred.append("real")
    else:
        y_pred.append("fake")

In [ ]:
# creo i vettori di ground truth
y_true = np.array(["fake"] * len(fake_images))
temp = np.array(["real"] * len(real_images))
y_true = np.concatenate([y_true, temp])

# calcolo la matrice di confusione 
cm = confusion_matrix(y_true, y_pred, labels=["real", "fake"])
print(cm)

In [ ]:
tn, fp, fn, tp = cm.ravel()

# metriche
accuracy = round((tp + tn) / (tp + tn + fp + fn), 4) * 100
precision = round((tp) / (tp + fp), 4) * 100
recall = round((tp) / (tp + fn), 4) * 100
specificity = round((tn) / (tn + fp) * 100, 4)
f1_score = round((2 * precision * recall) / (precision + recall), 4)

print({"Accuracy":accuracy, "Precision":precision, "Recall":recall, "Specificity":specificity, "F1 Score":f1_score})

In [ ]:
bin_y_true = []
bin_y_pred = []

for t in y_true:
    if t == "fake":
        bin_y_true.append(1)
    else:
        bin_y_true.append(0)


for p in y_pred:
    if p == "fake":
        bin_y_pred.append(1)
    else:
        bin_y_pred.append(0)

aps = average_precision_score(bin_y_true, bin_y_pred)

print("Average precision score: " + aps)